## 7.1 데커레이터 기본 지식

- 데커레이터는 다른 함수를 인수로 받는 콜러블이다.

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner # deco()가 inner() 함수 객체를 반환

@deco
def target(): # target()을 deco로 데커레이트
    print('running target()')
    
target() # 실제로는 inner()를 실행

running inner()


In [2]:
target # target이 inner()를 가리키고 있음

<function __main__.deco.<locals>.inner()>

## 7.2 파이썬이 데커레이터를 실행하는 시점

In [3]:
registry = []

def register(func):
    print('runing register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__=='__main__':
    main()

runing register(<function f1 at 0x0000024969809550>)
runing register(<function f2 at 0x0000024969809EE0>)
running main()
registry -> [<function f1 at 0x0000024969809550>, <function f2 at 0x0000024969809EE0>]
running f1()
running f2()
running f3()


In [4]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), 'py'))

In [5]:
import registration

runing register(<function f1 at 0x0000024969809430>)
runing register(<function f2 at 0x0000024969809940>)


In [6]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

## 7.3 데커레이터로 개선한 전략 패턴

In [7]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidality >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체의 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다"""
    return max(promo(order) for promo in promos)

## 7.4 변수 범위 규칙

In [9]:
def f1(a):
    print(a)
    print(b)

In [10]:
f1(3)

3


NameError: name 'b' is not defined

In [11]:
b = 6
f1(3)

3
6


In [12]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [13]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [14]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [15]:
f3(3)

3
6


In [16]:
b

9

In [17]:
f3(3)

3
9


In [18]:
from dis import dis

In [19]:
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [20]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## 7.5 클로저

In [22]:
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [23]:
avg = Averager()
avg(10)

10.0

In [24]:
avg(15)

12.5

In [25]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value) # 리스트가 가변형이어서 가능
        total = sum(series)
        return total/len(series)
    
    return averager

In [26]:
avg = make_averager()
avg(10)

10.0

In [27]:
avg(15)

12.5

In [28]:
avg.__code__.co_varnames

('new_value', 'total')

In [29]:
avg.__code__.co_freevars

('series',)

In [31]:
avg.__closure__ # cell 객체

(<cell at 0x0000024969848A30: list object at 0x00000249698294C0>,)

In [33]:
avg.__closure__[0].cell_contents

[10, 15]

## 7.6 nonlocal 선언

In [34]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1 # count라는 지역 변수를 만든다, count가 더 이상 자유 변수가 아니므로 클로저에 저장되지 않음
        total += new_value
        return total / count
    
    return averager

In [35]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [63]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total # global은 왜 쓰면 안 되지?
        count += 1 # count라는 지역 변수를 만든다, count가 더 이상 자유 변수가 아니므로 클로저에 저장되지 않음
        total += new_value
        return total / count
    
    return averager

In [64]:
avg = make_averager()
avg(10)

10.0

## 7.7 간단한 데커레이터 구현하기

In [60]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked